In [2]:
source extend_bashkernel-2modes.source
set-global-evaluation-mode "my-script"

Changing global mode to 'my-script'


In [ ]:
set-global-evaluation-mode "provided-script"

# RPM 生成方法

## はじめに

rpm作成手順を学ぶためには、最小構成のrpmを作ってみる事が一番の近道です。本章では、実際に最小構成rpmを作りながらrpm作成の基礎を学んで行きます。

今回、演習用のサンプルプログラムでは、rpmファイルを作成するためのspecファイル(tiny_web_example/rpmbuild/SPECS/tiny-web-example.spec)はあらかじめ準備されています。以下のセクションでは、そのspecファイルをスクラッチから作る方法を紹介しています。そのため、今回の演習用に、rpmファイルを生成するだけであれば、「2 rpmbuild環境の構築」完了後に、サンプルプログラムのspecファイルを使ってrpmファイルを生成してみてください。

## 演習について

* コマンドは全てsshを利用してJenkinsが稼働するマシンで実行します。
* スクリプト例に記載されているコードは実際に実行できます。課題をどう解決したら良いか解らなくなったら参考にして下さい。

# rpmbuild環境の構築

## 演習 : 必要なパッケージをインストールする

### 演習の内容

rpmビルドに必要なパッケージをインストールします。以下のパッケージが必要です。
* git
* rpm-build
* rpmlint

インストールにはyumコマンドを利用します。
```
$ sudo yum install -y git rpm-build rpmlint
```
実行結果例：

> ```
> $ sudo yum install -y git rpm-build rpmlint
> Loaded plugins: fastestmirror
> Setting up Install Process
> Determining fastest mirrors
>  * base: ftp.riken.jp
>  * extras: ftp.riken.jp
>  * updates: ftp.riken.jp
> ...(省略)...
> Complete!
> ```

**次のセルで、パッケージのインストールを実行して下さい。**

### スクリプト例

In [ ]:
# Task: create-rpm-install-packages
# Evaluation Mode: provided-script

# Install required packages

ssh -qi ../mykeypair root@10.0.2.100 "sudo yum install -y git rpm-build rpmlint"

### 確認
次のセルを実行して、必要なパッケージのインストールが完了していることを確認して下さい。

In [ ]:
# Task: create-rpm-install-packages
# Evaluation Mode: check

## 演習 : rpmbuildディレクトリを作成する

### 演習の内容

rpmbuildコマンドが使用するディレクトリを作成します。
rpmbuildコマンドを動作させるには、特定のディレクトリ構造が必要です。
次のディレクトリを作成します。
```
${HOME}/rpmbuild/BUILD
${HOME}/rpmbuild/BUILDROOT
${HOME}/rpmbuild/RPMS
${HOME}/rpmbuild/SOURCES
${HOME}/rpmbuild/SPECS
${HOME}/rpmbuild/SRPMS
```

ディレクトリの作成には``mkdir``コマンドを使用します。
```
$ mkdir -p \
 ${HOME}/rpmbuild/{BUILD,BUILDROOT,RPMS,SOURCES,SPECS,SRPMS}
```

**次のセルで、ディレクトリを作成して下さい。**

### スクリプト例

In [ ]:
# Task: create-rpm-create-folders
# Evaluation Mode: provided-script

# Create the folders used by rpmbuild

ssh -qi ../mykeypair root@10.0.2.100 'mkdir -p ${HOME}/rpmbuild/{BUILD,BUILDROOT,RPMS,SOURCES,SPECS,SRPMS}'

### 確認
次のセルを実行して、ディレクトリ生成が完了していることを確認して下さい。

In [ ]:
# Task: create-rpm-create-folders
# Evaluation Mode: check

# example-0.1.0.rpmの作成

## rpm作成へ向けた情報整理

「何のrpmを作るのか？」は「exampleのrpmを作る」と言う所までは決めていました。しかし、基になるパッケージや、そのバージョンが決まっていません。これらの未決定情報を整理して行きます。

### パッケージ情報の整理

rpmを作るには、事前に幾つかの情報を整理しておく必要があります。

1. パッケージ名
2. パッケージのバージョン

今回は、以下のようにします。

1. パッケージ名：`example`
2. バージョン：`0.1.0`

すると、期待するファイル名は`example-0.1.0.rpm`と言う事になります。

### 必須ファイルの準備

rpmを作るには、2つのファイルが必要です。

1. rpmspecファイル
2. ソースコードのアーカイブファイル

本書では、コンパイル不要なもソースコードを使用します。コンパイルが必要なソースコードをビルドするには、ビルド手順を記述する必要があります。これは、どちらかと言うと応用編です。最小構成rpm作りを目的としている本書では、触れません。

2つのファイルを作成して行きます。

## 演習 : rpmspecファイルを作成する

### 演習の内容

rpmspecには2つのルールがあります。

1. rpmspecファイルの配置場所は、${HOME}/rpmbuild/SPECS
2. rpmspecファイル名は、パッケージ名.spec

これを踏まえてrpmspecファイルを作成して行きます。

rpmspecファイルは、example.specと言う事になります。ここで事前に整理した情報を使います。
* パッケージ名：example
* バージョン：0.1.0

上記情報をrpmspecのタグで読み替えます。
* Name: example
* Version: 0.1.0

**example.sepc**
```
Name:    example
Version: 0.1.0
Release: 1%{?dist}
Summary: example
License: BSD
Source:  %{name}-%{version}.tar.gz

%description

%prep
%setup -q

%build

%install
rm   -rf $RPM_BUILD_ROOT
mkdir -p $RPM_BUILD_ROOT
rsync -avx ./ $RPM_BUILD_ROOT/

%clean
rm -rf $RPM_BUILD_ROOT

%files
%defattr(-,root,root,-)
/
%doc

%changelog```


**<項目の補足説明>**

%install
* アーカイブツリー全てを仮想ルートツリーへ配置します

%files
* /配下全てをパッケージング対象に入れます


**次のセルで、example.sepcを作成して下さい。**

**[ヒント]** リモートでのファイル作成には、ヒアドキュメントを利用します。

### スクリプト例

In [ ]:
# Task: create-rpm-create-spec-file
# Evaluation Mode: provided-script

# Create the spec file

ssh -qi ../mykeypair root@10.0.2.100 <<'EOS'
cat <<'EOF' > ${HOME}/rpmbuild/SPECS/example.spec
Name:    example
Version: 0.1.0
Release: 1%{?dist}
Summary: example
License: BSD
Source:  %{name}-%{version}.tar.gz

%description

%prep
%setup -q

%build

%install
rm   -rf $RPM_BUILD_ROOT
mkdir -p $RPM_BUILD_ROOT
rsync -avx ./ $RPM_BUILD_ROOT/

%clean
rm -rf $RPM_BUILD_ROOT

%files
%defattr(-,root,root,-)
/
%doc

%changelog
EOF
EOS

### 確認
次のセルを実行して、rpmspecファイル（example.sepc）の生成が完了していることを確認して下さい。

In [ ]:
# Task: create-rpm-create-spec-file
# Evaluation Mode: check

## 演習 : rpmsepcファイルの内容を確認する

### 演習の内容

作成したrpmspecファイルに必須項目が含まれているかどうかをrpmlintコマンドで確認します。
もしもrpmspecファイルに問題がある場合は、rpmlintコマンドが警告してくれます。

実行結果例：
> ```
> $ rpmlint example.spec
> example.spec: W: no-buildroot-tag
> example.spec: W: invalid-url Source0: example-0.1.0.tar.gz
> 0 packages and 1 specfiles checked; 0 errors, 2 warnings.
> ```

**次のセルで、rpmlintを実行して下さい。**

### スクリプト例

In [ ]:
# Task: create-rpm-check-required-fields
# Evaluation Mode: provided-script

# Chech required fields

ssh -qi ../mykeypair root@10.0.2.100 <<'EOF'
cd ${HOME}/rpmbuild/SPECS/
rpmlint example.spec
EOF

### Check

In [ ]:
# Task: create-rpm-check-required-fields
# Evaluation Mode: check
# (Uncertain if check is needed or not)

## 演習 : ソースコードファイルを配置する

### 演習の内容

次の手順でアーカイブするソースコードファイルをrpmbuild配下に配置します。

**ディレクトリ example-0.1.0 を作成します**
* アーカイブファイル名には規則性があり、`パッケージ名-バージョン`という名前のディレクトリをアーカイブした、`パッケージ名-バージョン.tar.gz`と言うファイル名にする必要があります。
* まず、`${HOME}/rpmbuild/SOURCES`配下に、名前が"パッケージ名-バージョン"のディレクトリを作成します。
```
$ cd ${HOME}/rpmbuild/SOURCES
$ mkdir example-0.1.0
```

**example-0.1.0 配下にアーカイブするソースコードファイルを配置します**
* 次にパッケージングするファイルを配置します。既存RPMと重複しないファイルであれば良いので、今回は`/bin/example`という空のファイルとします。
```
$ mkdir example-0.1.0/bin
$ touch example-0.1.0/bin/example
```

**ディレクトリ配下のファイル確認します**
```
$ find example-0.1.0 -type f
```

実行結果例：
> ```
> $ find example-0.1.0 -type f
> example-0.1.0/bin/example
> ```

これはrpm化された時に期待するファイルリストです。



**次のセルで、上記のアーカイブ手順を実行して下さい。**

**[ヒント]** リモートで複数のコマンドを実行するには、ヒアドキュメントを利用します。

### スクリプト例

In [ ]:
# Task: create-rpm-prepare-contents
# Evaluation Mode: provided-script

# Prepare the contents and create a package

ssh -qi ../mykeypair root@10.0.2.100 <<'EOF'
cd ${HOME}/rpmbuild/SOURCES
mkdir example-0.1.0
mkdir example-0.1.0/bin
touch example-0.1.0/bin/example
find example-0.1.0 -type f
EOF

### 確認
次のセルを実行して、ソースコードアーカイブの準備が完了していることを確認して下さい。

In [ ]:
# Task: create-rpm-prepare-contents
# Evaluation Mode: check

## 演習 : アーカイブファイルを作成する

### 演習の内容

tarコマンドを使用してアーカイブファイルを作成します。アーカイブファイル名は、必ず`パッケージ名-バージョン.tar.gz`とします。
```
$ tar zcvf example-0.1.0.tar.gz example-0.1.0
```

実行結果例：

> ```
> $ tar zcvf example-0.1.0.tar.gz example-0.1.0
> example-0.1.0/
> example-0.1.0/bin/
> example-0.1.0/bin/example
> ```

**次のセルで、アーカイブを作成して下さい。**

### スクリプト例

In [ ]:
# Task: create-rpm-create-tar
# Evaluation Mode: provided-script

# Prepare the contents and create a package

ssh -qi ../mykeypair root@10.0.2.100 <<'EOF'
cd ${HOME}/rpmbuild/SOURCES
tar zcvf example-0.1.0.tar.gz example-0.1.0
EOF

### 確認
次のセルを実行して、アーカイブファイルの作成が完了していることを確認して下さい。

In [ ]:
# Task: create-rpm-create-tar
# Evaluation Mode: check

## 演習 : rpmを作成する

### 演習の内容

`rpmbuild`コマンドを使ってrpmを作成します。

```
$ rpmbuild -bb example.spec
```

実行結果例：

> ```
> $ rpmbuild -bb example.spec
> Executing(%prep): /bin/sh -e /var/tmp/rpm-tmp.thSbon
> + umask 022
> + cd /home/vagrant/rpmbuild/BUILD
> + cd /home/vagrant/rpmbuild/BUILD
> + rm -rf example-0.1.0
> + /bin/tar -xf -
> + /usr/bin/gzip -dc /home/vagrant/rpmbuild/SOURCES/example-0.1.0.tar.gz
> + STATUS=0
> + '[' 0 -ne 0 ']'
> + cd example-0.1.0
> + /bin/chmod -Rf a+rX,u+w,g-w,o-w .
> + exit 0
> Executing(%build): /bin/sh -e /var/tmp/rpm-tmp.eYFskt
> + umask 022
> + cd /home/vagrant/rpmbuild/BUILD
> + cd example-0.1.0
> + exit 0
> Executing(%install): /bin/sh -e /var/tmp/rpm-tmp.cwEehz
> + umask 022
> + cd /home/vagrant/rpmbuild/BUILD
> + cd example-0.1.0
> + rm -rf /home/vagrant/rpmbuild/BUILDROOT/example-0.1.0-1.el6.x86_64
> + mkdir -p /home/vagrant/rpmbuild/BUILDROOT/example-0.1.0-1.el6.x86_64
> + rsync -avx ./ /home/vagrant/rpmbuild/BUILDROOT/example-0.1.0-1.el6.x86_64/
> sending incremental file list
> ./
> bin/
> bin/example
>
> sent 128 bytes  received 38 bytes  332.00 bytes/sec
> total size is 0  speedup is 0.00
> + /usr/lib/rpm/brp-compress
> + /usr/lib/rpm/brp-strip
> + /usr/lib/rpm/brp-strip-static-archive
> + /usr/lib/rpm/brp-strip-comment-note
> Processing files: example-0.1.0-1.el6.x86_64
> Requires(rpmlib): rpmlib(CompressedFileNames) <= 3.0.4-1 rpmlib(PayloadFilesHavePrefix) <= 4.0-1
> Checking for unpackaged file(s): /usr/lib/rpm/check-files /home/vagrant/rpmbuild/BUILDROOT/example-0.1.0-1.el6.x86_64
> warning: Could not canonicalize hostname: vagrant-centos6
> Wrote: /home/vagrant/rpmbuild/RPMS/x86_64/example-0.1.0-1.el6.x86_64.rpm
> Executing(%clean): /bin/sh -e /var/tmp/rpm-tmp.6TMilN
> + umask 022
> + cd /home/vagrant/rpmbuild/BUILD
> + cd example-0.1.0
> + rm -rf /home/vagrant/rpmbuild/BUILDROOT/example-0.1.0-1.el6.x86_64
> + exit 0
> ```

**次のセルで、rpmbuildを実行して下さい。**

### スクリプト例

In [ ]:
# Task: create-rpm-build-rpm
# Evaluation Mode: provided-script

# Build the rpm file

ssh -qi ../mykeypair root@10.0.2.100 <<'EOF'
cd ${HOME}/rpmbuild/SPECS
rpmbuild -bb example.spec
EOF

### 確認
次のセルを実行して、rpm作成が完了していることを確認して下さい。

In [ ]:
# Task: create-rpm-build-rpm
# Evaluation Mode: check

## 演習 : 出来上がったrpmをインストールする

### 演習の内容

**rpmコマンドでインストールします**
* rpmbuildの実行結果には、作成したrpmの配置場所が出力されています。
```
Wrote: /home/vagrant/rpmbuild/RPMS/x86_64/example-0.1.0-1.el6.x86_64.rpm 
```  
* rpmコマンドを使って、生成したrpmパッケージをインストールします。
```
$ sudo rpm -ivh /home/vagrant/rpmbuild/RPMS/x86_64/example-0.1.0-1.el6.x86_64.rpm
```
  
実行結果例：
> ```
> $ sudo rpm -ivh /home/vagrant/rpmbuild/RPMS/x86_64/example-0.1.0-1.el6.x86_64.rpm
> Preparing...                ########################################### [100%]
>    1:example                ########################################### [100%]
> ```


**インストールしたrpmの情報を確認します**
```
$ rpm -qi example
```

実行結果例：
> ```
> $ rpm -qi example
> Name        : example                      Relocations: (not relocatable)
> Version     : 0.1.0                             Vendor: (none)
> Release     : 1.el6                         Build Date: Wed 15 Apr 2015 07:20:32 PM JST
> Install Date: Wed 15 Apr 2015 07:21:59 PM JST      Build Host: vagrant-centos6
> Group       : Unspecified                   Source RPM: example-0.1.0-1.el6.src.rpm
> Size        : 0                                License: BSD
> Signature   : (none)
> Summary     : example
> Description :
>
> ```

**ファイルリストを確認します**
```
$ rpm -ql example
```

実行結果例：
> ```
> /
> /bin
> /bin/example
> ```

**実際にファイルが存在しているかどうかを確認します**
```
$ ls -l /bin/example
```

実行結果例：
> ```
> $ ls -l /bin/example
> -rw-r--r-- 1 root root 0 Apr 15 19:18 /bin/example
> ```


**次のセルで、上記のインストール手順を実行して下さい。**

### スクリプト例

In [ ]:
# Task: create-rpm-install-from-rpm
# Evaluation Mode: provided-script

# Install and confirm the package contents/information

ssh -qi ../mykeypair root@10.0.2.100 <<'EOF'
sudo rpm -ivh /root/rpmbuild/RPMS/x86_64/example-0.1.0-1.el6.x86_64.rpm
rpm -qi example
rpm -ql example
ls -l /bin/example
EOF

### 確認
次のセルを実行して、パッケージのインストールが完了していることを確認して下さい。

In [ ]:
# Task: create-rpm-install-from-rpm
# Evaluation Mode: check

### 演習 : rpmが削除出来ることを確認する

### 演習の内容

次に正しくrpmが削除出来る事を確認します。

**パッケージを削除します**

正常に削除した場合は何も出力されません。
```
$ sudo rpm -e example
```

実行結果例：
> ```
> $ sudo rpm -e example
> ```

**インストールされてない事を確認します** 
```
$ rpm -qi example
```

実行結果例：
> ```
> $ rpm -qi example
> package example is not installed
> ```

**`/bin/example`が存在しない事を確認します**
```
$ ls -l /bin/example
```

実行結果例：
> ```
> $ ls -l /bin/example
> ls: cannot access /bin/example: No such file or directory
> ```


**次のセルで、上記の削除手順を実行して下さい。**

### スクリプト例

In [ ]:
# Task: create-rpm-remove-package
# Evaluation Mode: provided-script

# Remove the package

ssh -qi ../mykeypair root@10.0.2.100 'sudo rpm -e example'

### 確認
次のセルを実行して、rpmの削除が完了していることを確認して下さい。

In [ ]:
# Task: create-rpm-remove-package
# Evaluation Mode: check

# まとめ

最小構成rpmを作ってrpm作成手順を学びました。一般的なアーカイブは、ビルド手順を定義する必要が大半です。インストールパスも様々です。そうしたパッケージ特有の手順に合わせて、rpmspecファイルの`%build`セクションと`%install`セクションを記述する必要がありますが、RPMを作る為の手順は、最小構成RPMと同じです。是非、最小構成RPM作成を発展させた独自パッケージ作成に挑戦してみて下さい。